сначала просто посмотрим как у нас уменьшается функция ошибок:

In [1]:
import numpy as np
import sys
sys.path.insert(0, "./Optimization-lib")
from lrs import *
from gradient_descent import GradientDecent, ScipyWrapper
from newton import NewtonCG
from function_wrapper import FunctionWrapper
from output import *
from graphics_plotter import GraphicsPlotter
from stochastic_gradient_descent import StochGradientDecent
from ucimlrepo import fetch_ucirepo

wine_quality = fetch_ucirepo(id=186)

X = wine_quality.data.features  # Признаки
y = wine_quality.data.targets

X = X.values
y = y.values.ravel()

def normalize(X_train):
    # 1. Вычисляем среднее и стандартное отклонение ТОЛЬКО на тренировочных данных
    mean = X_train.mean(axis=0)  # Среднее по каждому признаку
    std = X_train.std(axis=0)     # Стандартное отклонение по каждому признаку
    std[std == 0] = 1            # Избегаем деления на 0 (если std=0)

    # 2. Нормализуем тренировочные и тестовые данные
    X_train = (X_train - mean) / std
    # X_test = (X_test - mean) / std

    return X_train

X = normalize(X)
print(X)
print("...")
print(y)

def generate_weight_bounds(X, abs_bound: int) -> list:
    """
    Генерирует границы весов для всех параметров модели (включая intercept)
    n_features: Количество признаков (без учёта const)
    """
    n_features = X.shape[1]
    return [[-1 * abs_bound, abs_bound] for _ in range(n_features + 1)]  # +1 для intercept

def generate_start(X) -> list:
    n_features = X.shape[1]
    return [1 for _ in range(n_features + 1)]  # +1 для intercept


bounds = generate_weight_bounds(X, 100)
start = generate_start(X)


gradient.clear()
sgd = StochGradientDecent(exponential_decay(0.01, 0.0001), generate_weight_bounds(X, 1000), X, y, 5)
error_min = sgd.find_min(generate_start(X), 100)
print(error_min)

pretty_dataset_print(sgd, "EXP1", error_min, gradient)

/########################################################################


# добавить график изменения значения error_min от количества итераций (?)
#


[[ 0.14247327  2.18883292 -2.19283252 ...  1.81308951  0.19309677
  -0.91546416]
 [ 0.45103572  3.28223494 -2.19283252 ... -0.11507303  0.99957862
  -0.58006813]
 [ 0.45103572  2.55330026 -1.91755268 ...  0.25811972  0.79795816
  -0.58006813]
 ...
 [-0.55179227 -0.6054167  -0.88525328 ... -1.42124765 -0.47897144
  -0.91546416]
 [-1.32319841 -0.30169391 -0.12823371 ...  0.75571005 -1.016626
   1.9354021 ]
 [-0.93749534 -0.78765037  0.42232597 ...  0.25811972 -1.41986693
   1.09691202]]
...
[5 5 5 ... 6 7 6]
1.198636997179145

        EXP1
            found min error:         1.198637
            found weights:      ['5.21133720995793225939', '0.30234003274131970862', '0.02383398268353200886', '0.08005404352048683947', '0.34728576113367309874', '0.22506738678133056397', '0.07515526039064049368', '0.22231006827667096348', '-0.01820718985491492220', '0.19281433485607024636', '0.02411510112484734417', '0.83491347017108896722']
            steps count:          100
            function calls

TypeError: cannot unpack non-iterable int object

сейчас хотим выделять из dataset набор данных для обучения, и набор данных для проверки достоверности подобранных коэффициентов.



In [3]:
def custom_train_test_split(X, y, test_size=0.2, random_state=None):
    """просто рандомно делим на массивы"""
    if random_state is not None:
        np.random.seed(random_state)

    indices = np.arange(len(X))
    np.random.shuffle(indices)

    split_idx = int(len(X) * (1 - test_size))

    X_train = X[indices[:split_idx]]
    X_test = X[indices[split_idx:]]
    y_train = y[indices[:split_idx]]
    y_test = y[indices[split_idx:]]

    return X_train, X_test, y_train, y_test


X_train, X_test, y_train, y_test = custom_train_test_split(X, y, test_size=0.2, random_state=42)

bounds = generate_weight_bounds(X, 100)
start = generate_start(X)


gradient.clear()
sgd = StochGradientDecent(exponential_decay(0.01, 0.0001), generate_weight_bounds(X_train, 1000), X_train, y_train, 5)
error_min = sgd.find_min(generate_start(X_train), 120) # !! полезно менять max_steps
pretty_dataset_print(sgd, "EXP1", error_min, gradient)

# Получаем обученные веса
trained_weights = sgd.get_x()

# Прогнозируем на тестовой выборке
X_test = np.c_[np.ones(X_test.shape[0]), X_test] # это мы добавили столбец 1 что бы отдельно с константами не возиться (1, x1, x2...)
y_pred = np.dot(X_test, trained_weights)  # предсказание результата


# Вычисляем метрики качества на тестовой выборке
# сюда вскяие функции для анализа впихнуть и вывод написать что мы молодцы

print("\nРезультаты на тестовой выборке:")
print("True values:", y_test)
print("Predicted values:", y_pred)
#
def mean_squared_error(y_true, y_pred):
    return ((y_true - y_pred) ** 2).mean()

def mean_absolute_error(y_true, y_pred):
    return np.abs(y_true - y_pred).mean()

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

print(f"\nMSE: {mse:.4f}")
print(f"MAE: {mae:.4f}")
# print(f"R2 Score: {r2:.4f}")



34.14911758236231
33.46719931051104
31.69036359685944
30.72491711910142
29.360385580374768
28.758625337192736
27.386166895049666
25.748114018781568
24.017325865580133
22.95243948493021
21.571083707456904
20.73645722912144
19.66351043015703
19.13092337294149
18.7514478478786
18.07781424380001
17.369961398332123
17.144502714362396
16.38188018080983
15.685940870787501
15.190689623911148
14.603724152688256
14.286030268662486
13.62716700012163
13.235792287165147
12.352084264427926
11.446994757134892
11.188566701198077
10.703433247776756
10.528889009625331
10.00862504805125
9.717009934389843
9.467926177931012
9.315994605486267
8.602344581784171
8.511209889845002
7.97151670833524
7.915327322952725
7.121877131157623
6.713262441632635
6.59591221765655
6.331930957986341
6.172398228438836
5.877718135765888
5.7011118409588555
5.478212822777433
5.098067587598037
4.887402083170819
4.784232345926306
4.674494651977691
4.5408314045408
4.406433015861423
4.199758302816851
4.036455113435085
3.932124041452